In [1]:
import pandas as pd
import wandb
from tqdm.notebook import tqdm
import pickle
from os.path import exists
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
import ast
import scipy as sp
import scipy.stats as sps

from matplotlib.ticker import MaxNLocator
#...

font = {'family' : 'times',
        'size'   : 14}

matplotlib.rc('font', **font)

/Users/dehuang/.pyenv/versions/3.9.6/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
class Experiment:
    def __init__(self, run):
        self.name = run.name
        self.config = run.config
        self.summary = run.summary
        self.history = run.history()
        self.tags = run.tags
        self.run = run
        
    def get_id(self):
        return (self.config['formula'],self.config['mol_idx'])
        
    def get_history(self):
        return np.array(list(self.history['additional_steps'])).cumsum()

In [3]:
def fetch(project):
    api = wandb.Api()
    entity = "bogp"
    hdata = []
    runs = api.runs(entity + "/" + project)
    for run in tqdm(runs):
        try:
            hdata.append(Experiment(run))
        except:
            pass
    return hdata

In [4]:
raw = fetch("scale_master")

  0%|          | 0/298 [00:00<?, ?it/s]

In [5]:
exps = {}
for exp in raw:
    if exp.run.group == "bayes4":
        print(exp.name)
        exps[exp.config["num_particles"]] = exp
exps

transformer2_mswag_push_1_particles_1_devices
transformer2_mswag_push_64_particles_1_devices
transformer2_mswag_push_32_particles_1_devices
transformer2_mswag_push_8_particles_1_devices
transformer2_mswag_push_16_particles_1_devices
transformer2_mswag_push_4_particles_1_devices
transformer2_mswag_push_2_particles_1_devices


{1: <__main__.Experiment at 0x15fa23a30>,
 64: <__main__.Experiment at 0x15fa23910>,
 32: <__main__.Experiment at 0x15fa23b50>,
 8: <__main__.Experiment at 0x15fa23d90>,
 16: <__main__.Experiment at 0x15fa540a0>,
 4: <__main__.Experiment at 0x15fa545b0>,
 2: <__main__.Experiment at 0x15fa54eb0>}

In [6]:
def calc(p):
    d = ast.literal_eval(exps[p].history.orig_dist0[11])
    d_swag = ast.literal_eval(exps[p].history.max_dist0[len(exps[p].history.max_dist0) - 1])
    misclass = 0
    misclass_swag = 0
    for c in range(10):
        misclass += sum(d[c]) - d[c][c]
        misclass_swag += sum(d_swag[c]) - d_swag[c][c]
        # print("Orig", c, sps.entropy([x / sum(d[c]) for x in d[c]]))
        # print("Swag", c, sps.entropy([x / sum(d_swag[c]) for x in d_swag[c]]))
    print(misclass, misclass_swag)

In [7]:
calc(1)

220 236


In [8]:
calc(2)

238 249


In [9]:
calc(4)

215 196


In [10]:
calc(8)

209 163


In [11]:
calc(16)

221 166


In [12]:
calc(32)

ValueError: malformed node or string: None